# 프로젝트: 퍼스널 컬러 진단 - KNN 베이스코드

## 데이터
1. Cifer10 동물데이터: http://www.cs.toronto.edu/~kriz/cifar.html  
2. 직접 수집한 쿨톤 100장, 웜톤 100장  

## 목표
1. 우선 CS231에 나온 내용을 기반으로 cifer10 데이터셋을 KNN으로 분류하는 작업을 해보도록 하겠다!
2. 1단계를 적용하여 우리의 퍼스널컬러 데이터를 KNN으로 분류하는 작업을 해보자!

# 1. Cifer10 분류
## (1) 라이브러리 및 데이터 풀러오기

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# CIFAR-10 데이터셋 로드
(Xtr, Ytr), (Xte, Yte) = tf.keras.datasets.cifar10.load_data()

# 데이터 확인
print(Xtr.shape)  # (50000, 32, 32, 3)
print(Ytr.shape)  # (50000, 1)
print(Xte.shape)   # (10000, 32, 32, 3)
print(Yte.shape)   # (10000, 1)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [ ]:
print(Xtr)

[[[[ 59  62  63]
   [ 43  46  45]
   [ 50  48  43]
   ...
   [158 132 108]
   [152 125 102]
   [148 124 103]]

  [[ 16  20  20]
   [  0   0   0]
   [ 18   8   0]
   ...
   [123  88  55]
   [119  83  50]
   [122  87  57]]

  [[ 25  24  21]
   [ 16   7   0]
   [ 49  27   8]
   ...
   [118  84  50]
   [120  84  50]
   [109  73  42]]

  ...

  [[208 170  96]
   [201 153  34]
   [198 161  26]
   ...
   [160 133  70]
   [ 56  31   7]
   [ 53  34  20]]

  [[180 139  96]
   [173 123  42]
   [186 144  30]
   ...
   [184 148  94]
   [ 97  62  34]
   [ 83  53  34]]

  [[177 144 116]
   [168 129  94]
   [179 142  87]
   ...
   [216 184 140]
   [151 118  84]
   [123  92  72]]]


 [[[154 177 187]
   [126 137 136]
   [105 104  95]
   ...
   [ 91  95  71]
   [ 87  90  71]
   [ 79  81  70]]

  [[140 160 169]
   [145 153 154]
   [125 125 118]
   ...
   [ 96  99  78]
   [ 77  80  62]
   [ 71  73  61]]

  [[140 155 164]
   [139 146 149]
   [115 115 112]
   ...
   [ 79  82  64]
   [ 68  70  55]
   [ 67  69

In [ ]:
print(Ytr)

[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]


## (2) 데이터 전처리

In [ ]:
# 모든 이미지가 1차원 배열로 저장된다.
Xtr_rows = Xtr.reshape(Xtr.shape[0], 32 * 32 * 3) # Xtr_rows는 50000 x 3072 크기의 배열.
Xte_rows = Xte.reshape(Xte.shape[0], 32 * 32 * 3) # Xte_rows는 10000 x 3072 크기의 배열.

## (2) train, validation 분류하기

In [ ]:
# Xtr_rows, Ytr, Xte_rows, Yte 는 이전과 동일하게 갖고 있다고 가정하자.
# Xtr_rows 는 50,000 x 3072 행렬이었다.
Xval_rows = Xtr_rows[:1000, :] # 앞의 1000 개를 검증용으로 선택한다.
Yval = Ytr[:1000]
Xtr_rows = Xtr_rows[1000:, :] # 뒤쪽의 49,000 개를 학습용으로 선택한다.
Ytr = Ytr[1000:]

## (2) KNN(최접근이웃)

In [ ]:
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # nearest neighbor 분류기는 단순히 모든 학습 데이터를 기억해둔다.
        self.Xtr = X
        self.ytr = y

    def predict(self, X, k):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        # 출력 type과 입력 type이 갖게 되도록 확인해준다.
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

    # loop over all test rows
        for i in range(num_test):
        # i번째 테스트 이미지와 가장 가까운 학습 이미지를
        # L1 거리(절대값 차의 총합)를 이용하여 찾는다.
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            #distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis = 1))
            #min_index = np.argmin(distances) # 가장 작은 distance를 갖는 인덱스를 찾는다.
            #pred[i] = self.ytr[min_index] # 가장 가까운 이웃의 라벨로 예측

            indices = np.argsort(distances)  # 거리가 작은 순서대로 인덱스 정렬
            k_nearest_labels = self.ytr[indices[:k]]  # k개의 가장 가까운 이웃의 라벨
            unique, counts = np.unique(k_nearest_labels, return_counts=True)
            Ypred[i] = unique[np.argmax(counts)]  # 가장 많은 투표를 받은 라벨로 예측
        return Ypred

In [ ]:
# 검증 셋에서 가장 잘 동작하는 hyperparameter 들을 찾는다.
validation_accuracies = []
for k in [1, 3, 5, 10, 20, 50, 100]:

    # 특정 k 값을 정해서 검증 데이터에 대해 평가할 때 사용한다.
    nn = NearestNeighbor()
    nn.train(Xtr_rows, Ytr)
    # 여기서는 k를 input으로 받을 수 있도록 변형된 NearestNeighbor 클래스가 있다고 가정하자.
    Yval_predict = nn.predict(Xval_rows, k = k)
    acc = np.mean(Yval_predict == Yval)
    print('accuracy: %f' % (acc,))

    # 검증 셋에 대한 정확도를 저장해 놓는다.
    validation_accuracies.append((k, acc))

accuracy: 0.099237
accuracy: 0.099692
accuracy: 0.099476
accuracy: 0.099453
accuracy: 0.099738
accuracy: 0.099670
accuracy: 0.099609


In [ ]:
print(validation_accuracies)

[(1, 0.099237), (3, 0.099692), (5, 0.099476), (10, 0.099453), (20, 0.099738), (50, 0.09967), (100, 0.099609)]


-> 데이터 갯수에 비해 k값을 너무 작게 설정한 것 같아서 다시 수정해서 시도

In [ ]:
# 검증 셋에서 가장 잘 동작하는 hyperparameter 들을 찾는다.
validation_accuracies = []
for k in [100, 500, 2000, 4000, 8000]:

    # 특정 k 값을 정해서 검증 데이터에 대해 평가할 때 사용한다.
    nn = NearestNeighbor()
    nn.train(Xtr_rows, Ytr)
    # 여기서는 k를 input으로 받을 수 있도록 변형된 NearestNeighbor 클래스가 있다고 가정하자.
    Yval_predict = nn.predict(Xval_rows, k = k)
    acc = np.mean(Yval_predict == Yval)
    print('accuracy: %f' % (acc,))

    # 검증 셋에 대한 정확도를 저장해 놓는다.
    validation_accuracies.append((k, acc))

accuracy: 0.099609
accuracy: 0.100084
accuracy: 0.100414
accuracy: 0.100554
accuracy: 0.100498


In [ ]:
print(validation_accuracies)

[(100, 0.099609), (500, 0.100084), (2000, 0.100414), (4000, 0.100554), (8000, 0.100498)]


# 2. 퍼스널 컬러 분류

## (1) 라이브러리 및 데이터 불러오기

In [51]:
import tensorflow as tf
import numpy as np
import glob
from PIL import Image

In [52]:
def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기

    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)

    return images

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
# 모든 이미지 불러오기
directory_path = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/쿨톤"  # 디렉토리 경로
directory_path2 = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/웜톤"  # 디렉토리 경로
images = load_images_from_directory(directory_path)
images2 = load_images_from_directory(directory_path)
# 이미지 리스트 합치기
images = np.concatenate((images, images2), axis=0)

In [135]:
print(len(images))

100


## (2) 데이터 전처리

In [127]:
# print(images[0].shape)
# print(images[1].shape)
# print(images[2].shape)
# print(images[3].shape)
# print(images[4].shape)
# print(images[5].shape)
# print(images[6].shape)
# print(images[7].shape)
# print("-----------")

In [143]:
import numpy as np
import cv2

# 새로운 크기로 리사이즈할 이미지의 너비와 높이
new_width = 200
new_height = 200

resized_images = []
for image in images:
    resized_image = cv2.resize(image, (new_width, new_height))
    resized_images.append(resized_image)

# 결과 출력
#for resized_image in resized_images:
#    print(resized_image.shape)

# resized_images 리스트를 NumPy 배열로 변환
resized_images_array = np.array(resized_images)
# 잘되다가 갑자기 오류!!!!!!11
# 변환된 NumPy 배열 출력
#print(resized_images_array.shape)


<ipython-input-143-48ba09885a61>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  resized_images_array = np.array(resized_images)


ValueError: ignored

In [140]:
import numpy as np

# 리스트에 요소들이 포함되어 있다고 가정
image_list = [np.random.random((300, 300, 3)) for _ in range(100)]

# 리스트 전체를 NumPy 배열로 변환
image_array = np.array(image_list)

# 변환된 NumPy 배열 출력
print(image_array)


[[[[0.4772414  0.68029003 0.08391245]
   [0.83936054 0.47281372 0.00476186]
   [0.64742671 0.32642495 0.90297943]
   ...
   [0.51383562 0.59496876 0.72701242]
   [0.73023395 0.78817072 0.45996244]
   [0.0238904  0.74286733 0.59785902]]

  [[0.48601732 0.06861294 0.57160267]
   [0.78236027 0.13188145 0.45318939]
   [0.85533714 0.28817663 0.65792491]
   ...
   [0.75517398 0.69007279 0.57664914]
   [0.47016104 0.38640689 0.10719906]
   [0.10223663 0.66243484 0.11615291]]

  [[0.56509641 0.32724237 0.12356815]
   [0.917994   0.88334095 0.70861392]
   [0.33803571 0.77700248 0.58670597]
   ...
   [0.26757932 0.85268579 0.04465357]
   [0.72655417 0.43319128 0.97651091]
   [0.6184965  0.80118245 0.49629465]]

  ...

  [[0.25003767 0.98366053 0.87566698]
   [0.48018024 0.731978   0.54409864]
   [0.40160597 0.24288041 0.19762659]
   ...
   [0.25635708 0.66589214 0.80763129]
   [0.35743469 0.82447232 0.59252447]
   [0.64217604 0.83513045 0.96032679]]

  [[0.50860394 0.44324446 0.78117593]
   [0.7

In [ ]:
images = resized_images_array.reshape(resized_images_array.shape[0], 400 * 400 * 3)

In [ ]:
# # 수정 필요
# # 모든 이미지가 1차원 배열로 저장된다.
# re_images = []
# for i in range(8):
#   shape = images[i].shape
#   length_1, length_2, length_3 = shape[0], shape[1], shape[2]
#   print(length_1, length_2, length_3)
#   images_rows = images[i].reshape(images[i].shape[0], shape[0] * shape[1] * shape[2]) # images_rows 크기
#   re_images.append(images_rows)

In [136]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 처음 50개의 요소를 1로 변경
labels[50:] = 1

# 0이 쿨톤 1이 웜톤
print(len(images))
print(labels)

100
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


## (3) train, val, test 분리하기

In [67]:
def split_data(data, labels, train_ratio, val_ratio):
    # 데이터 개수 확인
    num_examples = len(data)
    num_lables = len(labels)
    print("총 데이터 갯수: ", len(data))
    print("총 라벨 갯수: ", len(labels))

    # 인덱스를 섞은 배열 생성
    shuffled_indices = np.random.permutation(num)

    # 훈련 세트, 검증 세트, 테스트 세트 크기 계산
    train_size = int(num_examples * train_ratio)
    val_size = int(num_examples * val_ratio)
    test_size = num_examples - train_size - val_size

    # 데이터와 레이블을 분할하여 저장할 변수 초기화
    train_data = np.zeros((train_size, *data.shape[1:]))
    train_labels = np.zeros((train_size, *labels.shape[1:]))
    val_data = np.zeros((val_size, *data.shape[1:]))
    val_labels = np.zeros((val_size, *labels.shape[1:]))
    test_data = np.zeros((test_size, *data.shape[1:]))
    test_labels = np.zeros((test_size, *labels.shape[1:]))

    # 데이터와 레이블을 섞은 인덱스를 기반으로 분할
    train_indices = shuffled_indices[:train_size]
    val_indices = shuffled_indices[train_size:train_size+val_size]
    test_indices = shuffled_indices[train_size+val_size:]

    train_data = data[train_indices]
    train_labels = labels[train_indices]
    val_data = data[val_indices]
    val_labels = labels[val_indices]
    test_data = data[test_indices]
    test_labels = labels[test_indices]

    return train_data, train_labels, val_data, val_labels, test_data, test_labels

In [107]:
train_ratio = 0.7  # 훈련 세트 비율
val_ratio = 0.2    # 검증 세트 비율
train_data, train_labels, val_data, val_labels, test_data, test_labels = split_data(images, labels, train_ratio, val_ratio)
print("train: ", train_data.shape)
print("val: ", val_data.shape)
print("test: ", test_data.shape)

총 데이터 갯수:  16
총 라벨 갯수:  16
train:  (11, 480000)
val:  (3, 480000)
test:  (2, 480000)


## (4) KNN(최접근이웃)

In [75]:
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # nearest neighbor 분류기는 단순히 모든 학습 데이터를 기억해둔다.
        self.Xtr = X
        self.ytr = y

    def predict(self, X, k):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        # 출력 type과 입력 type이 갖게 되도록 확인해준다.
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

    # loop over all test rows
        for i in range(num_test):
        # i번째 테스트 이미지와 가장 가까운 학습 이미지를
        # L1 거리(절대값 차의 총합)를 이용하여 찾는다.
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            #distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis = 1))
            #min_index = np.argmin(distances) # 가장 작은 distance를 갖는 인덱스를 찾는다.
            #pred[i] = self.ytr[min_index] # 가장 가까운 이웃의 라벨로 예측

            indices = np.argsort(distances)  # 거리가 작은 순서대로 인덱스 정렬
            k_nearest_labels = self.ytr[indices[:k]]  # k개의 가장 가까운 이웃의 라벨
            unique, counts = np.unique(k_nearest_labels, return_counts=True)
            Ypred[i] = unique[np.argmax(counts)]  # 가장 많은 투표를 받은 라벨로 예측
        return Ypred

In [108]:
# 검증 셋에서 가장 잘 동작하는 hyperparameter 들을 찾는다.
validation_accuracies = []
for k in [1, 3, 5, 10, 20]:

    # 특정 k 값을 정해서 검증 데이터에 대해 평가할 때 사용한다.
    nn = NearestNeighbor()
    nn.train(train_data, train_labels)
    # 여기서는 k를 input으로 받을 수 있도록 변형된 NearestNeighbor 클래스가 있다고 가정하자.
    Yval_predict = nn.predict(val_data, k = k)
    acc = np.mean(Yval_predict == val_labels)
    print('accuracy: %f' % (acc,))

    # 검증 셋에 대한 정확도를 저장해 놓는다.
    validation_accuracies.append((k, acc))

accuracy: 0.000000
accuracy: 0.000000
accuracy: 0.333333
accuracy: 0.666667
accuracy: 0.000000


In [89]:
print(validation_accuracies)

[(1, 0.0), (3, 0.0), (5, 0.0), (10, 0.0), (20, 0.0)]


## (5) skin detection 된 값으로 평균값을 넣으면 다를까?